<a href="https://colab.research.google.com/github/Kiro02/CUB-final-pjs/blob/main/DTSA5798FP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
from sklearn.preprocessing import LabelEncoder


# read data
df = pd.read_json('/Users/kiro02/coursera/DTSA5798.json')

df.head()

/Users/kiro02/coursera/ktrain/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


In [ ]:
# data preprocessing
df['headline'] = df['headline'].apply(lambda x: x.lower())
df['headline'] = df['headline'].str.replace('[^\w\s]', '')
df['headline'] = df['headline'].str.replace('\d+', '')
df['headline'] = df['headline'].str.replace('\n', '')
df['headline'] = df['headline'].str.replace(' +', ' ')


# label encoding
df['category'] = df['category'].apply(lambda x: 1 if x in [
    'HEALTHY LIVING', 'WELLNESS'
] else 0)
label_encoder = LabelEncoder()
df['category'] = label_encoder.fit_transform(df['category'])

In [ ]:
# split data
train_df, valid_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42
)

In [ ]:
import ktrain
from ktrain import text

# Load and preprocess the data
train_data, val_data, preproc = text.texts_from_df(
    train_df=train_df,
    text_column='headline',
    label_columns='category',
    val_df=valid_df,
    maxlen=512,
    preprocess_mode='distilbert'
)

# Unpack the train and validation data
(x_train, y_train) = (train_data.x, train_data.y)
(x_valid, y_valid) = (val_data.x, val_data.y)

['not_category', 'category']
        not_category  category
66880            1.0       0.0
68387            1.0       0.0
100514           1.0       0.0
197218           1.0       0.0
144020           0.0       1.0
['not_category', 'category']
        not_category  category
56274            1.0       0.0
9929             1.0       0.0
163271           1.0       0.0
190013           1.0       0.0
13564            1.0       0.0
preprocessing train...
language: en
train sequence lengths:
	mean : 10
	95percentile : 15
	99percentile : 17


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 10
	95percentile : 15
	99percentile : 17


In [ ]:
# Create the text classifier
model = text.text_classifier(
    name='distilbert',
    train_data=train_data,
    preproc=preproc
)

# Wrap the model and data in a Learner object
learner = ktrain.get_learner(
    model=model,
    train_data=train_data,
    val_data=val_data,
    batch_size=100
)

# Train the model
learner.fit_onecycle(
    lr=2e-5,
    epochs=1
)


Is Multi-Label? False
maxlen is 512
done.


begin training using onecycle policy with max lr of 2e-05...
1607/1607 [==============================] - 47042s 29s/step - loss: 0.1972 - accuracy: 0.9209 - val_loss: 0.1471 - val_accuracy: 0.9395


In [ ]:
# Evaluate the model
learner.validate(class_names=['NOT', 'Health&Wellness'])

# Save the model
predictor = ktrain.get_predictor(learner.model, preproc)

1256/1256 [==============================] - 417s 332ms/step
                 precision    recall  f1-score   support

            NOT       0.96      0.97      0.97     35353
Health&Wellness       0.77      0.71      0.74      4818

       accuracy                           0.94     40171
      macro avg       0.86      0.84      0.85     40171
   weighted avg       0.94      0.94      0.94     40171

